# Problemas de incidência

Uma versão particular dos problemas de alocação são os *problemas de incidência*. Genericamente estes problemas estão ligados aos conjuntos que é possível  formar com os elementos de um dado universo finito.

É frequente usar nestes problemas uma matriz binária $A$ com a seguinte semântica:

> $A_{i,j} = 1 \quad$ se e só se $\quad$ o elemento $i$ do universo está contido no conjunto $j$.

Estas matrizes chamam-se *matrizes de incidência* e daí resulta o nome genérico para estes problemas.

## Programação Inteira em Z3

Para utilizar o Z3 como solver de programação inteira (restrições em aritmética linear inteira mais critério de optimização), deve utilizar o construtor `Optimize` para criar uma instância vazia de um solver (em vez do construtor `Solver`). Para definir o critério de optimização deve utilizar os métodos `minimize` ou `maximize`, passando como parâmetro a função objectivo. Por exemplo, para maximizar a função $x + y$ sujeita às restrições $x < 2$ e $y - x < 1$ podemos usar o Z3 da seguinte forma.

In [1]:
from z3 import *

solver = Optimize()
x = Int("x")
y = Int("y")
solver.add(x < 2)
solver.add(y - x < 1)
solver.maximize(x+y)
print(solver.check())
print(solver.model())

sat
[y = 1, x = 1]


Se quisermos minimizar a mesma função objectivo com as mesmas restrições, temos um problema de optimização *unbounded*. Para termos a certeza se é este o caso, devemos usar os métodos `lower` ou `upper` para confirmar os valores mínimos ou máximos encontrados para a função objectivo. Programaticamente podemos testar se estes valores são diferentes de infinito usando, por exemplo, a função `is_int_value`.

In [2]:
solver = Optimize()
x = Int("x")
y = Int("y")
solver.add(x < 2)
solver.add(y - x < 1)
obj = solver.minimize(x+y)
print(solver.check())
print(solver.model())
print(solver.lower(obj))
print(is_int_value(solver.lower(obj)))

sat
[y = 0, x = 1]
-1*oo
False


## Set cover

Um dos problemas clássicos das ciências da computação nesta categoria é o problema do *set cover* descrito em https://en.wikipedia.org/wiki/Set_cover_problem.

Neste problema, são dados
- $U$, o universo de valores
- $S$, o conjunto de conjuntos,  cuja união é igual a $U$

O objectivo é determinar o menor número de conjuntos de $S$ cuja união é igual a $U$, i.e., a *cobertura mínima* de $U$.

### Análise do problema

Podemos representar este problema por uma matriz de incidência $A \in \{0,1\}^{|U| \times |S|}$. Por exemplo, se $U=\{0,1,2,3,4\}$ e $S = \{\{0,1,2\},\{1,3\},\{2,3\},\{3,4\}\}$ temos a seguinte matriz de incidência

$$
\begin{array}{c|c|c|c|c}
& S_0 & S_1 & S_2 & S_3\\
\hline
0 & 1 & 0 & 0 & 0\\
\hline
1 & 1 & 1 & 0 & 0\\
\hline
2 & 1 & 0 & 1 & 0\\
\hline
3 & 0 & 1 & 1 & 1\\
\hline
4 & 0 & 0 & 0 & 1
\end{array}
$$

Neste exemplo, a cobertura mínima é $S_0 \cup S_3 = U$.

Este problema pode ser resolvido com programação inteira usando uma variável inteira binária $x_j$ para cada conjunto $S_j$, que irá determinar se esse conjunto pertence à cobertura mínima. O objectivo é minimizar $\sum_j x_j$ obedecendo à seguinte restrição:
- Cada elemento de $U$ tem que pertencer a pelo menos um conjunto da cobertura mínima.

### Exercício 1
Formalize a restrição acima indicada.

$$
\forall_{i \in U} \cdot \sum_{j\in S} a_{i,j}\cdot x_{j} \ge 1 
$$

### Exercício 2

Usando o Z3, implemente a função `set_cover` que dada a matriz de incidência (representada como uma lista de colunas) determine quais os conjuntos que pertencem à cobertura mínima. 

In [6]:
def set_cover(A):
    s = Optimize()
    # completar 
    S=len(A)
    U=len(A[0])
    x={}
    for j in range(S):
        x[j]=Int(str(j))
        s.add(x[j]>=0,x[j]<=1)
    for i in range(U):
        s.add(sum([A[j][i]*x[j] for j in range(S)])>=1)
    s.minimize(Sum([x[j] for j in range(S)]))
    if s.check()==sat:
        m=s.model()
        return [j for j in range(S) if m[x[j]]==1]
    
    
   
            
assert set_cover([[1,1,1,0,0],[0,1,0,1,0],[0,0,1,1,0],[0,0,0,1,1]]) == [0,3]

## Bin packing

Outro problema clássico na categoria dos problemas de incidência, que generaliza o problema anterior, é o problema de empacotamento *bin packing* descrito em https://en.wikipedia.org/wiki/Bin_packing_problem. 

Neste problema, são dados
- $N$, o número de items a empacotar
- $C$, a capacidade das contentores onde pretendemos empacotar os items
- $W_i$ o peso de cada item $i$, com $0 < W_i \le C$

Pretende-se determinar o número mínimo de contentores necessários para empacotar todos os items (note que, no pior caso, tal será possível com $N$ contentores).

Por exemplo, se tivermos
- $N = 7$
- $C = 10$
- $W_0 = 2, W_1 = 5, W_2 = 4, W_3 = 7, W_4 = 1, W_5 = 3, W_6 = 8$

o número mínimo de contentores necessários é 3.

### Análise do problema

Ao contrário do problema anterior, em que a matriz de incidência é dada como input, neste problema pretende-se precisamente descobrir esta matriz, minimizando simultaneamente o número de contentores. Como tal, para resolver este problema com programação inteira iremos usar as seguintes variáveis:

- Uma matriz $A$ de variáveis binárias de dimensão $|N| \times |N|$, onde a variável $A_{i,j}$ determina se o item $i$ é colocado no contentor $j$
- Uma variável binária $y_j$ por cada contentor $j$ que determina se esse contentor é utilizado

O objectivo é minimizar $\sum_j y_j$ obedecendo às seguintes restrições:
- Cada item tem que ser empacotado num contentor
- A capacidade de cada contentor não pode ser excedida

### Exercício 3
Formalize as duas restrições acima indicadas.

 $$
 \forall_{i \in N} \cdot \sum_{j \in N} A_{i,j}=1 
 \\
 \forall_{j \in N} \cdot \sum_{i \in N} w_{i} \cdot A_{i,j} \leq C \cdot y_{j}
 $$

### Exercício 4

Implemente a função `binpacking` que dada a capacidade dos contentores e uma lista com os pesos dos items a empacotar, determine o número mínimo de contentores necessários para o fazer.

In [16]:
def binpacking(C,W):
    s = Optimize()
    # completar
    N=len(W)
    y={}
    A={}
    
    for j in range(N):
        y[j]=Int(str(j))
        s.add(y[j]>=0,y[j]<=1)
        for i in range(N):
            A[i,j]=Int(str(i)+','+str(j))
            s.add(A[i,j]>=0,A[i,j]<=1)
            
    for i in range(N):
        s.add(sum([A[i,j] for j in range(N)])==1)
        
    for j in range(N):
        s.add(Sum([W[i]*A[i,j] for i in range(N)])<=C * y[j])
        
    s.minimize(Sum([y[j] for j in range(N)]))
    if s.check() == sat:
        m = s.model()
        return m.eval(Sum([y[j] for j in range(N)]))



assert binpacking(10,[2,5,4,7,1,3,8]) == 3

### Exercício 5
Modifique a sua implementação da função anterior por forma a devolver uma lista com os identificadores dos contentores onde cada item deve ser empacotado.

In [22]:
def binpacking(C,W):
    s = Optimize()
    # completar
    N=len(W)
    y={}
    A={}
    
    for j in range(N):
        y[j]=Int(str(j))
        s.add(y[j]>=0,y[j]<=1)
        for i in range(N):
            A[i,j]=Int(str(i)+','+str(j))
            s.add(A[i,j]>=0,A[i,j]<=1)
            
    for i in range(N):
        s.add(sum([A[i,j] for j in range(N)])==1)
        
    for j in range(N):
        s.add(Sum([W[i]*A[i,j] for i in range(N)])<=C * y[j])
        
    s.minimize(Sum([y[j] for j in range(N)]))
    l=[]
    if s.check() == sat:
        m = s.model()
        return [[[i for i in range(N) if m[A[i,j]]==1]] for j in range(N) if m[y[j]]==1]

binpacking(10,[2,5,4,7,1,3,8])

[[[1, 2, 4]], [[0, 6]], [[3, 5]]]

## Knapsack

Outro problema clássico de incidência é o *problema da mochila* descrito em https://en.wikipedia.org/wiki/Knapsack_problem.

Neste problema, são dados
- $N$, o número de items disponíveis
- $C$, a capacidade da mochila
- $W_i$, o peso de cada item $i$, com $0 < W_i \le C$
- $V_i$ o valor de cada item $i$, com $0 \le V_i$

Pretende-se determinar o valor máximo que pode ser transportado na mochila.

Por exemplo, se tivermos uma mochila com capacidade $C = 15$ e 5 items com os seguintes pesos e valores
$$
\begin{array}{c|c|c}
& W & V\\
\hline
0 & 12 & 4\\
1 & 2 & 2\\
2 & 1 & 2\\
3 & 1 & 1\\
4 & 4 & 10
\end{array}
$$
a melhor solução é empacotar todos os items menos o primeiro, com um valor total de 15.

### Exercício 6
Formalize este problema usando programação inteira.

Pode usar-se uma variável binária $x_i$ por cada item, para decidir se esse item será colocado na mochila.

Maximiza-se $\sum_i x_i \cdot V_i$ sujeito à seguinte restrição, que garante que a capacidade da mochila não é excedida.

$$
\sum_{i = 0}^{N-1} x_i \cdot W_i \le C
$$

### Exercício 7
Implemente a função `knapsack` que dada a capacidade da mochila e uma lista com um par *(peso, valor)* por cada item, determine quais os items a empacotar na mochila e o respectivo valor.

In [23]:
def knapsack(C,I):
    s = Optimize()
    
    N = len(I)
    x = {}
    
    for i in range (N):
        x[i] = Int (str(i))
        s.add(x[i]>= 0 , x[i]<= 1)
    
    s.add(sum([x[i] * I[i][0]  for i in range(N)]) <= C)
    
    s.maximize(sum([x[i] * I[i][1]  for i in range(N)]))
    
    if s.check() == sat:
        m = s.model()
        return [i for i in range(N) if (m[x[i]] == 1)] 
    
assert knapsack(15,[(12,4),(2,2),(1,2),(1,1),(4,10)]) == [1,2,3,4]